
Logging levels in Python

DEBUG: Detailed information, typcically of interest only when diagnosing problems

INFO: Confirmation that things are working as expected

WARNING: An indication that something unexpected happened, 
or indicative of some problem in the near future eg: Disk space filled but software still working

ERROR: Due to serious problem, the software is not able to perform some of the function 

Critical: Entire program itself not able to run.


In [1]:
def adding(x,y):
    return x + y

num1 = 10
num2 = 5
print('Addition of {} + {} = {}'.format(num1,num2,adding(num1,num2)))

Addition of 10 + 5 = 15


In [2]:
# Now we can replace the print with logging module

import logging

def adding(x,y):
    return x + y

num1 = 10
num2 = 5
logging.debug('Addition of {} + {} = {}'.format(num1,num2,adding(num1,num2)))

The above code didn't add any log to the console because default runtime logging level in python is warning and higher 

In [4]:
#Now lets change to warning
import logging

def adding(x,y):
    return x + y

num1 = 10
num2 = 5
logging.warning('Addition of {} + {} = {}'.format(num1,num2,adding(num1,num2)))

In [7]:
#Lets configure Logging in Python
# logging.DEBUG resolves to the integer 10 , Info 20, Warn 30, Error 40 and Critical 50
# https://docs.python.org/3/library/logging.html#logging-levels

import logging

logging.basicConfig(level=logging.DEBUG)  # Changing the logging level to DEBUG for this script

def adding(x,y):
    return x + y

num1 = 10
num2 = 5
logging.debug('Addition of {} + {} = {}'.format(num1,num2,adding(num1,num2)))

In [8]:
# To create a log file 

import logging

logging.basicConfig(filename='myscript.log', level=logging.DEBUG)  # Creates myscript.log file and appends the log

def adding(x,y):
    return x + y

num1 = 10
num2 = 5
logging.debug('Addition of {} + {} = {}'.format(num1,num2,adding(num1,num2)))

https://docs.python.org/3/library/logging.html#logrecord-attributes

In [9]:
# Lets add Log Record attributes

import logging

logging.basicConfig(filename='myscript.log', level=logging.DEBUG, 
                   format='%(asctime)s:%(levelname)s:%(message)s')  # Adding Log format option

def adding(x,y):
    return x + y

num1 = 10
num2 = 5
logging.debug('Addition of {} + {} = {}'.format(num1,num2,adding(num1,num2)))

Advanced Logging

- While importing several modules then there may be a different logger being set on that module
- However we can create our own logger and configure our own logger as follows

In [10]:
# This is File A - addition.py 

import logging

logging.basicConfig(filename='sample.log', level=logging.DEBUG, 
                   format='%(asctime)s:%(levelname)s:%(message)s')  # Adding Log format option

def adding(x,y):
    return x + y

num1 = 10
num2 = 5
logging.debug('Addition of {} + {} = {}'.format(num1,num2,adding(num1,num2)))

2021-06-27 20:45:33,315:root:Addition of 10 + 5 = 15

In [11]:
# This is File B - employee.py

import logging

logging.basicConfig(filename='employee.log', level=logging.INFO, 
                   format='%(levelname)s:%(name)s:%(message)s')  # Adding Log format option

class Employee:
    
    def __init__(self, first, last):
        self.first = first
        self.last = last
        
        logging.info('Created Employee: {} - {}'.format(self.fullname, self.email))
    
    @property
    def email(self):
        return '{}.{}@email.com'.format(self.first, self.last)
    
    @property
    def fullname(self):
        return '{} {}'.format(self.first, self.last)

emp1 = Employee('Saran', 'Raj')
emp2 = Employee('Sudeep', 'Sunthankar')

INFO:root:Created Employee: Saran Raj - Saran.Raj@email.com

INFO:root:Created Employee: Sudeep Sunthankar - Sudeep.Sunthankar@email.com

In [14]:
# Now there are two different loggers and while importing a module then the logger also get imported 
# however we can configure 

# 2021-06-27 20:45:33,315:root:Addition of 10 + 5 = 15  ( root indicates logger level - root logger)

# Here we are importing employee.py in the addition.py and 
# when we import the employee.py then the root logger created in employee.py will be used in the current file 
# Hence the sample.log in the addition.py will not be created, instead it creates only employee.log file as its root logger
# This is the reason, that creating root logger is not a recommended approach 
# because it appends the logs of both addition.py and employee.py in the employee.log 


import logging
import employee


logging.basicConfig(filename='sample.log', level=logging.DEBUG, 
                   format='%(asctime)s:%(levelname)s:%(message)s')  # Adding Log format option

def adding(x,y):
    return x + y

num1 = 10
num2 = 5
logging.debug('Addition of {} + {} = {}'.format(num1,num2,adding(num1,num2)))

In [15]:
# To avoid the above issue, we need to create a seperate logger for each module

# Creating a seperate logger in employee.py

import logging

empLogger = logging.getLogger(__name__) # Creating a logger for employee.py
empLogger.setLevel(logging.DEBUG)

# Specifying filename and formatter in the basicConfig indicates that its a root logger log file so we are creating our own fileHandler

formatter = logging.Formatter('%(levelname)s:%(name)s:%(message)s') # Creating our own custom formatter
file_handler = logging.FileHandler('employee.log')  # FileHandler meant for custom log file dedicated to this logger 
file_handler.setFormatter(formatter)
empLogger.addHandler(file_handler)

"""
logging.basicConfig(filename='employee.log', level=logging.INFO, 
                   format='%(levelname)s:%(name)s:%(message)s')  

Now this root logger is no longer needed as we created our custom
"""

class Employee:
    
    def __init__(self, first, last):
        self.first = first
        self.last = last
        
        empLogger.info('Created Employee: {} - {}'.format(self.fullname, self.email))
    
    @property
    def email(self):
        return '{}.{}@email.com'.format(self.first, self.last)
    
    @property
    def fullname(self):
        return '{} {}'.format(self.first, self.last)

emp1 = Employee('Saran', 'Raj')
emp2 = Employee('Sudeep', 'Sunthankar')

employee.log contains the following output If we execute the employee.py

INFO:__main__:Created Employee: Saran Raj - Saran.Raj@email.com
INFO:__main__:Created Employee: Sudeep Sunthankar - Sudeep.Sunthankar@email.com

However if employee.py is imported on someother 

INFO:employee:Created Employee: Saran Raj - Saran.Raj@email.com
INFO:employee:Created Employee: Sudeep Sunthankar - Sudeep.Sunthankar@email.com


addition.log will be created and it contains the following log after importing the above employee.log

2021-06-27 20:45:33,315:root:Addition of 10 + 5 = 15


In [1]:
# Lets create custom logger for the addition.py as well

import logging

additionLogger = logging.getLogger(__name__) # Creating a logger for addition.py
additionLogger.setLevel(logging.DEBUG)
# Specifying filename and formatter in the basicConfig indicates that its a root logger log file so we are creating our own fileHandler

formatter = logging.Formatter('%(asctime)s:%(levelname)s:%(message)s') # Creating our own custom formatter
file_handler = logging.FileHandler('addition.log')  # FileHandler meant for custom log file dedicated to this logger 
file_handler.setFormatter(formatter)
additionLogger.addHandler(file_handler)

"""
logging.basicConfig(filename='sample.log', level=logging.DEBUG, 
                   format='%(asctime)s:%(levelname)s:%(message)s')  # Adding Log format option
"""

def adding(x,y):
    return x + y

num1 = 10
num2 = 5
additionLogger.debug('Addition of {} + {} = {}'.format(num1,num2,adding(num1,num2)))

addition.log would look like the following

2021-06-27 20:45:33,315:__main__:Addition of 10 + 5 = 15

In [ ]:
# We can also set the log level at the file_handler level which will override the logger level 

import logging

additionLogger = logging.getLogger(__name__) # Creating a logger for addition.py
additionLogger.setLevel(logging.DEBUG)
# Specifying filename and formatter in the basicConfig indicates that its a root logger log file so we are creating our own fileHandler

formatter = logging.Formatter('%(asctime)s:%(levelname)s:%(message)s') # Creating our own custom formatter
file_handler = logging.FileHandler('addition.log')  # FileHandler meant for custom log file dedicated to this logger 
file_handler.setFormatter(formatter)

file_handler.setLevel(logging.ERROR)

additionLogger.addHandler(file_handler)

"""
logging.basicConfig(filename='sample.log', level=logging.DEBUG, 
                   format='%(asctime)s:%(levelname)s:%(message)s')  # Adding Log format option
"""

def adding(x,y):
    return x + y

num1 = 10
num2 = 5
additionLogger.debug('Addition of {} + {} = {}'.format(num1,num2,adding(num1,num2)))

Note:
If you would like to capture the traceback exception in the logging then we can make use of 

logging.exception('caught exception message here')

In [ ]:
# There is an option to use Stream handler which will print the logs in the output console as well as writing it in the log file

import logging

additionLogger = logging.getLogger(__name__) # Creating a logger for addition.py
additionLogger.setLevel(logging.DEBUG)
# Specifying filename and formatter in the basicConfig indicates that its a root logger log file so we are creating our own fileHandler

formatter = logging.Formatter('%(asctime)s:%(levelname)s:%(message)s') # Creating our own custom formatter
file_handler = logging.FileHandler('addition.log')  # FileHandler meant for custom log file dedicated to this logger 
file_handler.setLevel(logging.ERROR)
file_handler.setFormatter(formatter)

stream_handler = logging.StreamHandler()
stream_handler.setFormatter(formatter)

additionLogger.addHandler(file_handler)
additionLogger.addHandler(stream_handler)

"""
logging.basicConfig(filename='sample.log', level=logging.DEBUG, 
                   format='%(asctime)s:%(levelname)s:%(message)s')  # Adding Log format option
"""

def adding(x,y):
    return x + y

num1 = 10
num2 = 5
additionLogger.debug('Addition of {} + {} = {}'.format(num1,num2,adding(num1,num2)))